# Augmented variables from set A

Imports

In [42]:
import os
import pandas as pd
import xgboost as xgb
from bayes_opt import BayesianOptimization
import sklearn.metrics as skmetric
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle

Get parent directoy:

In [79]:
def get_parent_dir(directory):
    import os
    return os.path.dirname(directory)

current_dirs_parent = get_parent_dir(os.getcwd())
dataraw_dir=current_dirs_parent+"/01.Data/Raw/"
preproc_dir=current_dirs_parent+"/01.Data/Preprocessing/"
models_a_dir=current_dirs_parent+"/03.Models/Set A/"
models_b_dir=current_dirs_parent+"/03.Models/Set B/"

Load data

In [3]:
train_a=pd.read_csv(preproc_dir+"train_a_preproc1.csv")
test_a=pd.read_csv(preproc_dir+"test_a_preproc1.csv")

In [4]:
train_a.shape

(17582, 1767)

In [5]:
test_a.shape

(1879, 1767)

Mean on failed states

In [6]:
train_a_failed=train_a.loc[train_a["y"]==1]

In [7]:
train_a_failed.head()

,time,y,x1,x2,x3,x4,x5,x6,x7,x8,...,x60_df2_l3,x61_df2_l3,xminute,xhour,xday,xmonth,xtimedif,xskip,xskipid,xsinceskip
259,1999-05-01 08:38:00,1,0.875829,0.180124,4.388169,321.833781,124.737513,292.598124,-3.373756,-3.413735,...,0.000001,1.0,38,8,1,5,0.0,0,1,260
260,1999-05-01 10:32:00,1,0.988543,4.594932,4.576621,-3.803762,125.132639,-1.441717,-3.373756,-3.413735,...,-0.000899,1.0,32,10,1,5,112.0,1,2,1
261,1999-05-01 10:34:00,1,0.990521,4.616294,4.576621,-3.803762,125.231760,-1.353542,-3.373756,-3.413735,...,-0.000899,1.0,34,10,1,5,0.0,0,2,2
262,1999-05-01 10:36:00,1,0.992498,4.594932,4.576621,-3.803762,125.330882,-1.265367,-3.373756,-3.413735,...,-0.000899,-1.0,36,10,1,5,0.0,0,2,3
263,1999-05-01 10:38:00,1,0.994476,4.594932,4.576621,5.958600,125.430018,1.574210,-3.373756,-3.413735,...,0.000899,-2.0,38,10,1,5,0.0,0,2,4


In [8]:
train_a_failed.tail()

,time,y,x1,x2,x3,x4,x5,x6,x7,x8,...,x60_df2_l3,x61_df2_l3,xminute,xhour,xday,xmonth,xtimedif,xskip,xskipid,xsinceskip
16828,1999-05-25 08:22:00,1,0.645837,18.117684,4.506428,224.054881,124.641566,292.793253,-3.433878,-3.323706,...,-1.000000e-06,0.0,22,8,25,5,0.0,0,14,23
16948,1999-05-25 12:22:00,1,0.307198,14.728827,17.909956,320.794260,125.374857,313.596354,-3.413735,-3.333777,...,-1.000000e-06,0.0,22,12,25,5,0.0,0,14,143
16949,1999-05-25 12:24:00,1,0.178031,14.797270,18.253865,321.189249,125.354411,319.607005,-3.413735,-3.333777,...,1.000000e-06,0.0,24,12,25,5,0.0,0,14,144
17282,1999-05-25 23:30:00,1,0.290296,14.485831,17.659005,293.298655,124.926859,289.025584,-3.493694,-3.383827,...,0.000000e+00,0.0,30,23,25,5,0.0,0,14,477
17283,1999-05-25 23:32:00,1,0.270459,14.450160,17.368878,293.197489,124.912684,291.573283,-3.484971,-3.383827,...,0.000000e+00,0.0,32,23,25,5,0.0,0,14,478


In [9]:
columns=train_a_failed.columns
selcols=[col for col in columns if col.startswith("x")]

X_failed=train_a_failed[train_a_failed.columns.intersection(selcols)].copy()

In [10]:
failed_mean=X_failed.mean()

In [11]:
failed_mean

x1              0.299776
x2              7.068634
x3              4.626460
x4            277.254285
x5            125.394050
                 ...    
xmonth          5.000000
xtimedif        1.220756
xskip           0.010554
xskipid         6.838171
xsinceskip    766.598945
Length: 1765, dtype: float64

In [12]:
failed_min=X_failed.min()

In [13]:
failed_min

x1             -3.803762
x2             -3.644716
x3             -2.814962
x4             -3.803762
x5            121.104266
                 ...    
xmonth          5.000000
xtimedif        0.000000
xskip           0.000000
xskipid         1.000000
xsinceskip      1.000000
Length: 1765, dtype: float64

In [14]:
failed_max=X_failed.max()

In [15]:
failed_max

x1               4.886031
x2              26.271027
x3              30.169444
x4             628.387156
x5             129.724940
                 ...     
xmonth           5.000000
xtimedif       832.000000
xskip            1.000000
xskipid         14.000000
xsinceskip    3752.000000
Length: 1765, dtype: float64

Xgboost trees on failed states

In [16]:
X_a_train=train_a[train_a.columns.intersection(selcols)].copy()
X_a_test=test_a[test_a.columns.intersection(selcols)].copy()

In [48]:
y_train_label=train_a["y"]
y_lead1_train_label=train_a["y"].shift(-1,fill_value=0)
y_lead2_train_label=train_a["y"].shift(-2,fill_value=0)

In [20]:
init=250
step=10
print(y_train_label[init:(init+step)],y_lead1_train_label[init:(init+step)],y_lead2_train_label[init:(init+step)])

250    0
251    0
252    0
253    0
254    0
255    0
256    0
257    0
258    0
259    1
Name: y, dtype: int64 250    0.0
251    0.0
252    0.0
253    0.0
254    0.0
255    0.0
256    0.0
257    0.0
258    1.0
259    1.0
Name: y, dtype: float64 250    0.0
251    0.0
252    0.0
253    0.0
254    0.0
255    0.0
256    0.0
257    1.0
258    1.0
259    1.0
Name: y, dtype: float64


In [26]:
X_a_train.fillna(0, inplace=True) 
X_a_train.replace(to_replace=np.inf, value=0, inplace=True)

X_a_test.fillna(0, inplace=True) 
X_a_test.replace(to_replace=np.inf, value=0, inplace=True)

In [27]:
scaler= StandardScaler().fit(X_a_train)
X_train_sc=scaler.transform(X_a_train)
X_test_sc=scaler.transform(X_a_test)

In [36]:
dtrain_lead2 = xgb.DMatrix(X_train_sc, label=y_train_label)
dtest_lead2 = xgb.DMatrix(X_train_sc)

C:\Users\jchas\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\jchas\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [37]:
def xgb_evaluate(max_depth, eta, subsample,colsample_bytree,gamma,lambda1,alpha):
    params = {'eval_metric': 'aucpr',
              'max_depth': int(max_depth),
              'subsample': subsample, #0.8
              'eta': eta, #0.3.. 0.2
              'gamma': gamma,
              'lambda': lambda1,
              'aplha': alpha,
              'colsample_bytree': colsample_bytree,
              'objective': 'binary:logistic',
              'verbosity': 0}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain_lead2, num_boost_round=100, nfold=5)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return cv_result['test-aucpr-mean'].iloc[-1]

xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 15),                                              
                                             'subsample': (0.3, 0.9),
                                             'eta': (0.3, 0.9),
                                             'gamma': (0.3, 0.9),
                                             'lambda1': (0.0, 1.0),
                                             'alpha': (0.0, 1.0),
                                             'colsample_bytree': (0.3, 1)})
    

xgb_bo.maximize(init_points=10, n_iter=100, acq='ei')


|   iter    |  target   |   alpha   | colsam... |    eta    |   gamma   |  lambda1  | max_depth | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.9702   |  0.7673   |  0.4459   |  0.8319   |  0.3628   |  0.6387   |  7.086    |  0.4268   |
|  2        |  0.9845   |  0.8556   |  0.6656   |  0.7755   |  0.8283   |  0.6997   |  13.17    |  0.8516   |
|  3        |  0.9845   |  0.5412   |  0.8283   |  0.302    |  0.3641   |  0.4121   |  5.537    |  0.5095   |
|  4        |  0.9848   |  0.7971   |  0.318    |  0.4568   |  0.3802   |  0.478    |  7.829    |  0.7011   |
|  5        |  0.9754   |  0.8306   |  0.3605   |  0.717    |  0.681    |  0.1171   |  3.848    |  0.5419   |
|  6        |  0.987    |  0.7181   |  0.6313   |  0.3024   |  0.875    |  0.2961   |  12.23    |  0.7137   |
|  7        |  0.9865   |  0.1823   |  0.7424   |  0.3302   |  0.679    |  0.8887   |  11.65    |  0.7262   |
|  8      

|  45       |  0.9881   |  0.7397   |  0.9649   |  0.3001   |  0.3469   |  0.9653   |  11.4     |  0.8952   |
|  46       |  0.9875   |  0.9825   |  0.4307   |  0.3054   |  0.4162   |  0.03917  |  11.27    |  0.8844   |
|  47       |  0.9878   |  0.02103  |  0.5238   |  0.3011   |  0.8821   |  0.4623   |  10.36    |  0.8995   |
|  48       |  0.9879   |  0.9078   |  0.4571   |  0.3006   |  0.8778   |  0.01513  |  12.06    |  0.8963   |
|  49       |  0.9873   |  0.9039   |  0.8189   |  0.3029   |  0.8064   |  0.9977   |  14.84    |  0.8984   |
|  50       |  0.9876   |  0.05187  |  0.9493   |  0.3031   |  0.3701   |  0.5098   |  12.16    |  0.8993   |
|  51       |  0.9878   |  0.9607   |  0.7631   |  0.3095   |  0.7445   |  0.00725  |  12.18    |  0.8968   |
|  52       |  0.9867   |  0.6059   |  0.4522   |  0.3001   |  0.3434   |  0.02997  |  11.26    |  0.8988   |
|  53       |  0.9871   |  0.02104  |  0.9992   |  0.3039   |  0.6391   |  0.6166   |  9.162    |  0.8968   |
|  54     

|  90       |  0.9872   |  0.9421   |  0.3417   |  0.301    |  0.666    |  0.9127   |  14.17    |  0.8828   |
|  91       |  0.9884   |  0.8955   |  0.3584   |  0.3062   |  0.5401   |  0.003648 |  14.74    |  0.8838   |
|  92       |  0.9867   |  0.05154  |  0.7579   |  0.3023   |  0.4168   |  0.1107   |  14.01    |  0.8987   |
|  93       |  0.9876   |  0.3563   |  0.3519   |  0.3074   |  0.3724   |  0.9415   |  14.23    |  0.8981   |
|  94       |  0.9878   |  0.06854  |  0.5355   |  0.303    |  0.3272   |  0.9315   |  13.97    |  0.8915   |
|  95       |  0.9874   |  0.01012  |  0.8661   |  0.3019   |  0.56     |  0.01173  |  14.48    |  0.8987   |
|  96       |  0.987    |  0.8496   |  0.9621   |  0.3047   |  0.3762   |  0.08974  |  13.49    |  0.8968   |
|  97       |  0.9874   |  0.9005   |  0.6466   |  0.3003   |  0.7501   |  0.9453   |  14.98    |  0.8997   |
|  98       |  0.987    |  0.05963  |  0.3753   |  0.3108   |  0.308    |  0.9997   |  10.2     |  0.896    |
|  99     

In [39]:
params = xgb_bo.max["params"]
params['max_depth'] = int(params['max_depth'])

# Train a new model with the best parameters from the search
model2 = xgb.train(params, dtrain_lead2, num_boost_round=500)

In [40]:
# Predict on testing and training set
y_pred = model2.predict(dtest_lead2)
y_train_pred = model2.predict(dtrain_lead2)

In [45]:
pickle.dump(model2, open(models_a_dir+"xgboost_y.pickle.dat", "wb"))

Xgboost trees previous to failed states

In [49]:
dtrain_lead1 = xgb.DMatrix(X_train_sc, label=y_lead1_train_label)
dtest_lead1 = xgb.DMatrix(X_train_sc)

C:\Users\jchas\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\jchas\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [50]:
def xgb_evaluate_lead1(max_depth, eta, subsample,colsample_bytree,gamma,lambda1,alpha):
    params = {'eval_metric': 'aucpr',
              'max_depth': int(max_depth),
              'subsample': subsample, #0.8
              'eta': eta, #0.3.. 0.2
              'gamma': gamma,
              'lambda': lambda1,
              'aplha': alpha,
              'colsample_bytree': colsample_bytree,
              'objective': 'binary:logistic',
              'verbosity': 0}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain_lead1, num_boost_round=100, nfold=5)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return cv_result['test-aucpr-mean'].iloc[-1]

xgb_bo_lead1 = BayesianOptimization(xgb_evaluate_lead1, {'max_depth': (3, 15),                                              
                                             'subsample': (0.3, 0.9),
                                             'eta': (0.3, 0.9),
                                             'gamma': (0.3, 0.9),
                                             'lambda1': (0.0, 1.0),
                                             'alpha': (0.0, 1.0),
                                             'colsample_bytree': (0.3, 1)})
    

xgb_bo_lead1.maximize(init_points=10, n_iter=100, acq='ei')


|   iter    |  target   |   alpha   | colsam... |    eta    |   gamma   |  lambda1  | max_depth | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.919    |  0.4771   |  0.4508   |  0.7609   |  0.7552   |  0.4277   |  12.83    |  0.8587   |
|  2        |  0.9094   |  0.5691   |  0.4643   |  0.5301   |  0.3541   |  0.02682  |  7.559    |  0.5575   |
|  3        |  0.9167   |  0.8543   |  0.6843   |  0.4729   |  0.6471   |  0.5617   |  6.962    |  0.4961   |
|  4        |  0.929    |  0.7508   |  0.4597   |  0.3049   |  0.6361   |  0.6719   |  9.549    |  0.6761   |
|  5        |  0.8925   |  0.5987   |  0.9065   |  0.8676   |  0.7631   |  0.8132   |  3.165    |  0.5982   |
|  6        |  0.9184   |  0.774    |  0.3344   |  0.4436   |  0.4251   |  0.5365   |  4.66     |  0.5956   |
|  7        |  0.9271   |  0.2998   |  0.5258   |  0.3745   |  0.3497   |  0.414    |  10.68    |  0.7484   |
|  8      

|  45       |  0.9252   |  0.04681  |  0.4482   |  0.324    |  0.8907   |  0.1764   |  11.12    |  0.8907   |
|  46       |  0.9319   |  0.00888  |  0.4886   |  0.3083   |  0.393    |  0.7417   |  11.58    |  0.8983   |
|  47       |  0.9296   |  0.4688   |  0.428    |  0.3106   |  0.4255   |  0.9785   |  13.15    |  0.8863   |
|  48       |  0.9144   |  0.9941   |  0.3837   |  0.3018   |  0.8737   |  0.7587   |  6.411    |  0.3499   |
|  49       |  0.9268   |  0.01835  |  0.7248   |  0.3143   |  0.3937   |  0.06923  |  5.739    |  0.8821   |
|  50       |  0.9237   |  0.04589  |  0.4409   |  0.3162   |  0.4749   |  0.9032   |  3.679    |  0.8557   |
|  51       |  0.9283   |  0.9498   |  0.4275   |  0.3019   |  0.6631   |  0.6915   |  9.683    |  0.8789   |
|  52       |  0.9261   |  0.3745   |  0.6258   |  0.3088   |  0.8718   |  0.4007   |  7.219    |  0.8924   |
|  53       |  0.9282   |  0.1387   |  0.3163   |  0.3179   |  0.8533   |  0.9396   |  13.06    |  0.8933   |
|  54     

|  90       |  0.9306   |  0.5403   |  0.5665   |  0.3174   |  0.4483   |  0.4788   |  10.09    |  0.8975   |
|  91       |  0.9313   |  0.7761   |  0.3207   |  0.3008   |  0.4125   |  0.9325   |  14.3     |  0.899    |
|  92       |  0.9295   |  0.0936   |  0.3104   |  0.3068   |  0.8201   |  0.9654   |  14.29    |  0.8962   |
|  93       |  0.9287   |  0.7831   |  0.5984   |  0.3047   |  0.5844   |  0.9997   |  14.35    |  0.8955   |
|  94       |  0.9265   |  0.7783   |  0.3826   |  0.3051   |  0.8213   |  0.5577   |  3.825    |  0.8968   |
|  95       |  0.9314   |  0.5583   |  0.3096   |  0.3115   |  0.4144   |  0.9902   |  13.99    |  0.8942   |
|  96       |  0.9286   |  0.9231   |  0.3676   |  0.3147   |  0.7757   |  0.8629   |  10.44    |  0.8988   |
|  97       |  0.9284   |  0.8281   |  0.3479   |  0.3041   |  0.7967   |  0.6904   |  14.89    |  0.8844   |
|  98       |  0.9308   |  0.09381  |  0.5751   |  0.3177   |  0.3195   |  0.9751   |  14.27    |  0.898    |
|  99     

In [51]:
params_lead1 = xgb_bo_lead1.max["params"]
params_lead1['max_depth'] = int(params_lead1['max_depth'])

# Train a new model with the best parameters from the search
model2_lead1 = xgb.train(params_lead1, dtrain_lead1, num_boost_round=500)

In [52]:
pickle.dump(model2_lead1, open(models_a_dir+"xgboost_y_lead1.pickle.dat", "wb"))

Xgboost two periods before

In [53]:
dtrain_lead2r = xgb.DMatrix(X_train_sc, label=y_lead2_train_label)
dtest_lead2r = xgb.DMatrix(X_train_sc)

C:\Users\jchas\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\jchas\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [54]:
def xgb_evaluate_lead2(max_depth, eta, subsample,colsample_bytree,gamma,lambda1,alpha):
    params = {'eval_metric': 'aucpr',
              'max_depth': int(max_depth),
              'subsample': subsample, #0.8
              'eta': eta, #0.3.. 0.2
              'gamma': gamma,
              'lambda': lambda1,
              'aplha': alpha,
              'colsample_bytree': colsample_bytree,
              'objective': 'binary:logistic',
              'verbosity': 0}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain_lead2r, num_boost_round=100, nfold=5)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return cv_result['test-aucpr-mean'].iloc[-1]

xgb_bo_lead2 = BayesianOptimization(xgb_evaluate_lead2, {'max_depth': (3, 15),                                              
                                             'subsample': (0.3, 0.9),
                                             'eta': (0.3, 0.9),
                                             'gamma': (0.3, 0.9),
                                             'lambda1': (0.0, 1.0),
                                             'alpha': (0.0, 1.0),
                                             'colsample_bytree': (0.3, 1)})
    

xgb_bo_lead2.maximize(init_points=10, n_iter=100, acq='ei')

|   iter    |  target   |   alpha   | colsam... |    eta    |   gamma   |  lambda1  | max_depth | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.8699   |  0.1452   |  0.3372   |  0.6494   |  0.4534   |  0.7699   |  14.47    |  0.5874   |
|  2        |  0.8827   |  0.6015   |  0.6878   |  0.654    |  0.8671   |  0.8397   |  7.488    |  0.8521   |
|  3        |  0.8845   |  0.3377   |  0.3126   |  0.6012   |  0.4405   |  0.5581   |  6.144    |  0.8169   |
|  4        |  0.8396   |  0.3307   |  0.5571   |  0.6023   |  0.6472   |  0.8598   |  3.316    |  0.4537   |
|  5        |  0.8576   |  0.8583   |  0.9901   |  0.8589   |  0.6322   |  0.7047   |  6.295    |  0.7452   |
|  6        |  0.8481   |  0.9802   |  0.338    |  0.5936   |  0.7842   |  0.9307   |  9.807    |  0.3849   |
|  7        |  0.8756   |  0.6441   |  0.6685   |  0.3949   |  0.51     |  0.9608   |  7.383    |  0.5345   |
|  8      

|  45       |  0.9052   |  0.1075   |  0.3448   |  0.3094   |  0.4055   |  0.9906   |  9.966    |  0.8986   |
|  46       |  0.8988   |  0.849    |  0.9867   |  0.3026   |  0.8388   |  0.8975   |  8.585    |  0.8863   |
|  47       |  0.9055   |  0.3854   |  0.3588   |  0.3013   |  0.8577   |  0.887    |  13.54    |  0.8962   |
|  48       |  0.9061   |  0.318    |  0.3362   |  0.3019   |  0.4127   |  0.03434  |  14.94    |  0.8723   |
|  49       |  0.9074   |  0.9972   |  0.3343   |  0.3051   |  0.4566   |  0.06219  |  12.48    |  0.8967   |
|  50       |  0.9053   |  0.5841   |  0.5131   |  0.3125   |  0.3188   |  0.545    |  13.6     |  0.8981   |
|  51       |  0.9016   |  0.7758   |  0.3125   |  0.302    |  0.6113   |  0.1951   |  11.65    |  0.8831   |
|  52       |  0.9068   |  0.9801   |  0.3934   |  0.3021   |  0.4783   |  0.4134   |  14.35    |  0.8934   |
|  53       |  0.9039   |  0.05041  |  0.3941   |  0.3082   |  0.8315   |  0.7862   |  8.826    |  0.8908   |
|  54     

|  91       |  0.8907   |  0.9499   |  0.8907   |  0.3062   |  0.831    |  0.05918  |  3.16     |  0.8833   |
|  92       |  0.9053   |  0.1448   |  0.3183   |  0.3102   |  0.3117   |  0.9626   |  11.44    |  0.8842   |
|  93       |  0.9057   |  0.4746   |  0.3114   |  0.3019   |  0.8897   |  0.3609   |  12.98    |  0.8974   |
|  94       |  0.9008   |  0.2656   |  0.8829   |  0.3226   |  0.3236   |  0.8961   |  5.652    |  0.8998   |
|  95       |  0.9059   |  0.05096  |  0.3427   |  0.316    |  0.417    |  0.06863  |  13.51    |  0.8987   |
|  96       |  0.8979   |  0.3291   |  0.3269   |  0.3136   |  0.8017   |  0.1403   |  5.353    |  0.8895   |
|  97       |  0.9048   |  0.1714   |  0.3056   |  0.3042   |  0.8809   |  0.9088   |  9.685    |  0.8934   |
|  98       |  0.9026   |  0.0614   |  0.4051   |  0.3173   |  0.32     |  0.1786   |  14.93    |  0.8876   |
|  99       |  0.9043   |  0.9976   |  0.3488   |  0.3013   |  0.5861   |  0.07851  |  14.63    |  0.8976   |
|  100    

In [55]:
params_lead2 = xgb_bo_lead2.max["params"]
params_lead2['max_depth'] = int(params_lead2['max_depth'])

# Train a new model with the best parameters from the search
model2_lead2 = xgb.train(params_lead2, dtrain_lead2r, num_boost_round=500)

In [56]:
pickle.dump(model2_lead2, open(models_a_dir+"xgboost_y_lead2.pickle.dat", "wb"))

In [57]:
train_b=pd.read_csv(preproc_dir+"train_b_preproc1.csv")
test_b=pd.read_csv(preproc_dir+"test_b_preproc1.csv")

In [58]:
X_b_train=train_b[train_b.columns.intersection(selcols)].copy()
X_b_test=test_b[test_b.columns.intersection(selcols)].copy()

In [59]:
X_b_train.fillna(0, inplace=True) 
X_b_train.replace(to_replace=np.inf, value=0, inplace=True)

X_b_test.fillna(0, inplace=True) 
X_b_test.replace(to_replace=np.inf, value=0, inplace=True)

In [60]:
X_b_train_sc=scaler.transform(X_b_train)
X_b_test_sc=scaler.transform(X_b_test)

In [61]:
dtrain_b = xgb.DMatrix(X_b_train_sc)
dtest_b = xgb.DMatrix(X_b_test_sc)

In [62]:
y_pred_train_b = model2.predict(dtrain_b)
y_pred_test_b = model2.predict(dtest_b)

In [63]:
y_lead1_pred_train_b = model2_lead1.predict(dtrain_b)
y_lead1_pred_test_b = model2_lead1.predict(dtest_b)

In [64]:
y_lead2_pred_train_b = model2_lead2.predict(dtrain_b)
y_lead2_pred_test_b = model2_lead2.predict(dtest_b)

In [70]:
xgb_out_train_b=pd.DataFrame({"xgb_y_prob" : y_pred_train_b,"xgb_ylead1_prob" : y_lead1_pred_train_b,"xgb_ylead2_prob" : y_lead2_pred_train_b})
xgb_out_test_b=pd.DataFrame({"xgb_y_prob" : y_pred_test_b,"xgb_ylead1_prob" : y_lead1_pred_test_b,"xgb_ylead2_prob" : y_lead2_pred_test_b})

In [73]:
xgb_out_train_b.to_csv(preproc_dir+"xgb_out_train_b.csv",index=False)
xgb_out_test_b.to_csv(preproc_dir+"xgb_out_test_b.csv",index=False)

XGB on dataset b

In [75]:
dtrain_b_y = xgb.DMatrix(X_b_train_sc,label=train_b["y_lead2"])

C:\Users\jchas\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\jchas\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [77]:
def xgb_evaluate_blead2(max_depth, eta, subsample,colsample_bytree,gamma,lambda1,alpha):
    params = {'eval_metric': 'aucpr',
              'max_depth': int(max_depth),
              'subsample': subsample, #0.8
              'eta': eta, #0.3.. 0.2
              'gamma': gamma,
              'lambda': lambda1,
              'alpha': alpha,
              'colsample_bytree': colsample_bytree,
              'objective': 'binary:logistic',
              'verbosity': 0}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain_b_y, num_boost_round=100, nfold=5)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return cv_result['test-aucpr-mean'].iloc[-1]

xgb_bo_blead2 = BayesianOptimization(xgb_evaluate_blead2, {'max_depth': (3, 15),                                              
                                             'subsample': (0.3, 0.9),
                                             'eta': (0.3, 0.9),
                                             'gamma': (0.3, 0.9),
                                             'lambda1': (0.0, 1.0),
                                             'alpha': (0.0, 1.0),
                                             'colsample_bytree': (0.3, 1)})
    

xgb_bo_blead2.maximize(init_points=10, n_iter=100, acq='ei')

|   iter    |  target   |   alpha   | colsam... |    eta    |   gamma   |  lambda1  | max_depth | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.00947  |  0.5715   |  0.7388   |  0.553    |  0.3322   |  0.08343  |  5.997    |  0.4917   |
|  2        |  0.00978  |  0.2156   |  0.6753   |  0.7376   |  0.3475   |  0.3254   |  9.31     |  0.4495   |
|  3        |  0.01512  |  0.5893   |  0.7426   |  0.8263   |  0.5298   |  0.1834   |  14.83    |  0.6564   |
|  4        |  0.01217  |  0.68     |  0.3583   |  0.8249   |  0.4366   |  0.02315  |  10.54    |  0.8274   |
|  5        |  0.01365  |  0.7685   |  0.9703   |  0.3951   |  0.6108   |  0.5602   |  3.187    |  0.5484   |
|  6        |  0.01149  |  0.2466   |  0.6492   |  0.6468   |  0.4246   |  0.08093  |  12.94    |  0.4871   |
|  7        |  0.02918  |  0.01125  |  0.8044   |  0.7527   |  0.4117   |  0.5893   |  7.538    |  0.4812   |
|  8      

|  45       |  0.01448  |  0.968    |  0.9741   |  0.3032   |  0.398    |  0.5236   |  5.53     |  0.8907   |
|  46       |  0.01068  |  0.9885   |  0.4552   |  0.3349   |  0.8906   |  0.2269   |  10.2     |  0.6619   |
|  47       |  0.01089  |  0.8951   |  0.9792   |  0.3381   |  0.3053   |  0.3612   |  13.76    |  0.3147   |
|  48       |  0.01287  |  0.9136   |  0.372    |  0.3124   |  0.3146   |  0.03775  |  3.002    |  0.3754   |
|  49       |  0.01182  |  0.0344   |  0.8434   |  0.3164   |  0.3643   |  0.8517   |  3.151    |  0.8978   |
|  50       |  0.009324 |  0.9833   |  0.9959   |  0.8527   |  0.8564   |  0.03201  |  10.13    |  0.7796   |
|  51       |  0.01269  |  0.9805   |  0.8971   |  0.8781   |  0.727    |  0.0407   |  4.327    |  0.8992   |
|  52       |  0.01439  |  1.898e-0 |  1.0      |  0.7854   |  0.9      |  1.0      |  15.0     |  0.9      |
|  53       |  0.01159  |  0.9954   |  0.3021   |  0.3249   |  0.3414   |  0.005544 |  12.59    |  0.7831   |
|  54     

|  91       |  0.01024  |  0.9719   |  0.416    |  0.8897   |  0.307    |  0.5808   |  14.86    |  0.8346   |
|  92       |  0.01842  |  0.7602   |  0.9857   |  0.3196   |  0.8761   |  0.952    |  14.21    |  0.766    |
|  93       |  0.01172  |  0.9305   |  0.9764   |  0.3044   |  0.3099   |  0.003039 |  4.507    |  0.3588   |
|  94       |  0.01322  |  0.1529   |  0.3473   |  0.3322   |  0.5838   |  0.996    |  4.001    |  0.8654   |
|  95       |  0.01757  |  0.319    |  0.9859   |  0.312    |  0.3169   |  0.07813  |  4.402    |  0.8981   |
|  96       |  0.01218  |  0.2533   |  0.9978   |  0.3489   |  0.8721   |  0.791    |  10.02    |  0.7876   |
|  97       |  0.01297  |  0.000636 |  0.3897   |  0.4259   |  0.3286   |  0.8695   |  5.638    |  0.8862   |
|  98       |  0.01275  |  0.07643  |  0.3856   |  0.3149   |  0.466    |  0.4084   |  14.85    |  0.8647   |
|  99       |  0.01567  |  0.891    |  0.3086   |  0.3054   |  0.3003   |  0.1099   |  4.457    |  0.8126   |
|  100    

In [78]:
params_blead2 = xgb_bo_blead2.max["params"]
params_blead2['max_depth'] = int(params_blead2['max_depth'])

# Train a new model with the best parameters from the search
model2_blead2 = xgb.train(params_blead2, dtrain_b_y, num_boost_round=500)

In [80]:
pickle.dump(model2_blead2, open(models_b_dir+"xgboost_y_blead2.pickle.dat", "wb"))

In [81]:
y_lead2_pred_train_b_own = model2_blead2.predict(dtrain_b)
y_lead2_pred_test_b_own = model2_blead2.predict(dtest_b)

In [87]:
xgb_out_train_b_own=pd.DataFrame({"xgb_y_prob" : y_pred_train_b,
                                  "xgb_ylead1_prob" : y_lead1_pred_train_b,
                                  #"xgb_ylead2_prob" : y_lead2_pred_train_b,
                                  "xgb_yblead2_prob" : y_lead2_pred_train_b_own})
xgb_out_test_b_own=pd.DataFrame({"xgb_y_prob" : y_pred_test_b,
                                 "xgb_ylead1_prob" : y_lead1_pred_test_b,
                                 #"xgb_ylead2_prob" : y_lead2_pred_test_b,
                                 "xgb_yblead2_prob" : y_lead2_pred_test_b_own})

In [88]:
xgb_out_train_b_own.to_csv(preproc_dir+"xgb_out_train_b_own.csv",index=False)
xgb_out_test_b_own.to_csv(preproc_dir+"xgb_out_test_b_own.csv",index=False)